# Города: выгрузка роликов по рекламодателю
Пример выгрузки рекламных роликов по рекламодателю

## Описание задачи и условий расчета
- Регион: Самара
- Период: 08.03.2023
- ЦА: Все 4+
- Место просмотра: дом
- Каналы: все каналы проекта TV Index
- Ролики: Рекламодатель - ВТБ; Блок распространение - сетевой, орибтальный; Ролик тип - ролик 
- Статистики: Rtg(000), Duration, Reach(000), Reach%

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)

# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)

# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники
Получим идентификаторы, которые будут использоваться для формирования условий расчета

In [ ]:
# В справочнике регионов найдем id региона Самара
cats.get_tv_region(name=['Самара'])

# Это id 6

In [ ]:
# В справочнике мест просмотра найдем идентификатор для домашнего смотрения
cats.get_tv_location(name=['дом'])

# Это id 1

In [ ]:
# В справочнике рекламодателей найдем идентификатор ВТБ
cats.get_tv_advertiser(name=['ВТБ'])

#Это id 3373

In [ ]:
# Обратимся к справочнику типов распространения блоков
cats.get_tv_breaks_distribution()

# Сетевой - N, Орбитальный - O

In [ ]:
# В словаре типов рекламы получим id нужного нам типа
cats.get_tv_ad_type(name=['ролик'])

# Для Ролик, это id 1

## Формирование задания
Зададим условия расчета

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'. Можно указать несколько периодов
date_filter = [('2023-03-08', '2023-03-08')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра: дом
location_filter = 'locationId = 1'

# Задаем каналы
company_filter = None

# Фильтр программ
program_filter = None

# Фильтр блоков: Блок распространение - сетевой, орибтальный
break_filter = 'breaksDistributionType IN (N,O)'

# Фильтр роликов: Рекламодатель - Сбер; Ролик тип - ролик
ad_filter = 'advertiserId = 3373 and adTypeId = 1'

# Указываем список срезов
slices = [
    'regionName', #Регион
    'tvCompanyName', #Телекомпания
    'advertiserName', #Рекламодатель
    'adName', #Ролик
    'adSpotId', #Ролик ID выхода, обязательный атрибут! 
    'researchDate', #Дата, обязательный атрибут! 
    'adStartTime', #Ролик время начала
    'adStandardDuration', #Ролик ожидаемая длительность
    'adDistributionTypeName', #Ролик распространение
]

# Указываем список статистик для расчета
statistics = ['Rtg000','Duration','Reach000','ReachPer']

# Задаем условия сортировки
sortings = None

# Задаем опции расчета
options = {
    "kitId": 3 #TV Index Cities
}

## Расчет задания
В одной задаче можно рассчитать данные только по одному региону. Сформируем словарь регионов, которые по очереди будем передавать на расчет. 

In [ ]:
# Раскомментируйте строки для тех регионов, которые должны участвовать в расчете

regions_dict = {
    #40: 'БАРНАУЛ',
    #18: 'ВЛАДИВОСТОК',
    #5: 'ВОЛГОГРАД',
    #8: 'ВОРОНЕЖ',
    #12: 'ЕКАТЕРИНБУРГ',
    #25: 'ИРКУТСК',
    #19: 'КАЗАНЬ',
    #45: 'КЕМЕРОВО',
    #23: 'КРАСНОДАР',
    #17: 'КРАСНОЯРСК',
    #1: 'МОСКВА',
    #4: 'НИЖНИЙ НОВГОРОД',
    #15: 'НОВОСИБИРСК',
    #21: 'ОМСК',
    #14: 'ПЕРМЬ',
    #9: 'РОСТОВ-НА-ДОНУ',
    6: 'САМАРА',
    #2: 'САНКТ-ПЕТЕРБУРГ',
    #10: 'САРАТОВ',
    #39: 'СТАВРОПОЛЬ',
    #3: 'ТВЕРЬ',
    #55: 'ТОМСК',
    #16: 'ТЮМЕНЬ',
    #20: 'УФА',
    #26: 'ХАБАРОВСК',
    #13: 'ЧЕЛЯБИНСК',
    #7: 'ЯРОСЛАВЛЬ'
}

### Посчитаем задания в цикле

> ⚠️ Для корректного расчета статистик по регионам, в методе `build_task` должны быть активированы параметры `add_city_to_basedemo_from_region` и `add_city_to_targetdemo_from_region`

In [ ]:
# Посчитаем задания в цикле
tasks = []
print("Отправляем задания на расчет")

# Для каждого региона формируем задание и отправляем на расчет
for reg_id, reg_name in regions_dict.items():
    
    #Передаем id региона в company_filter:
    init_company_filter = company_filter
    
    if company_filter is not None:
        company_filter = company_filter + f' AND regionId IN ({reg_id})'
        
    else:
        company_filter = f'regionId IN ({reg_id})'
      
    # Формируем задание для API TV Index в формате JSON
    task_json = mtask.build_simple_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                        daytype_filter=daytype_filter, company_filter=company_filter, 
                                        location_filter=location_filter, basedemo_filter=basedemo_filter, 
                                        targetdemo_filter=targetdemo_filter,program_filter=program_filter, 
                                        break_filter=break_filter, ad_filter=ad_filter, 
                                        slices=slices, statistics=statistics, sortings=sortings, options=options,
                                        add_city_to_basedemo_from_region=True,
                                        add_city_to_targetdemo_from_region=True)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}    
    tsk['task'] = mtask.send_simple_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')
    
    company_filter = init_company_filter
    
print(f"\nid: {[i['task']['taskId'] for i in tasks]}") 

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), to_lists=True)        
    results.append(df_result)
    print('.', end = '')
df = pd.concat(results)

# Приводим порядок столбцов в соответствие с условиями расчета
df = df[slices+statistics]

df

## Сохраняем в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('cities_01_advertiser_filter')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)